In [1]:
import cudaq
from cudaq import spin
import networkx as nx
import numpy as np
from typing import Optional
import pandas as pd


In [14]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * (number_of_qubits -1)

In [3]:
coreset_weights = np.random.random(number_of_qubits)
coreset_vectors = np.random.random((number_of_qubits,2))

In [4]:
def get_Hamil_variables(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    index_vals_temp: Optional[int] = None,
    new_df: Optional[pd.DataFrame] = None,
):
    """
    Generates the variables required for Hamiltonian

    Args:
        coreset_vectors: Coreset vectors
        coreset_weights: Coreset weights
        index_vals_temp: Index in the hierarchy
        new_df: new dataframe create for this problem,

    Returns:
       Graph, weights and qubits
    """
    if new_df is not None and index_vals_temp is not None:
        coreset_weights, coreset_vectors = get_cv_cw(coreset_vectors, coreset_weights, index_vals_temp)
    
    coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(
        coreset_vectors, coreset_weights, metric="dot"
    )
    qubits = len(G.nodes)

    return G, weights, qubits

def create_Hamiltonian_for_K2(G, qubits, weights: np.ndarray = None,add_identity=False):
    """
    Generate Hamiltonian for k=2

    Args:
        G: Problem as a graph
        weights: Edge weights
        nodes: nodes of the graph
        add_identity: Add identiy or not. Defaults to False.

    Returns:
        _type_: _description_
    """
    H = 0
#     if add_identity:
#         H = -1 * sum(weights) 
#     else:
#         H = 0 

    for i, j in G.edges():
        weight = G[i][j]["weight"]#[0]
        H += weight * (spin.z(i) * spin.z(j))
        
    return H
        
def gen_coreset_graph(
    coreset_vectors: np.ndarray,
    coreset_weights: np.ndarray,
    metric: str = "dot",
):
    """
    Generate a complete weighted graph using the provided set of coreset points

    Parameters
    ----------
    coreset_weights : ndarray
        np.Coreset weights in array format

    coreset_vectors : ndarray
        Data points of the coreset

    metric : str
        Choose the desired metric for computing the edge weights.
        Options include: dot, dist

    Returns
    -------
    coreset : List((weight, vector))
        The set of points used to construct the graph
    G : NetworkX Graph
        A complete weighted graph
    H : List((coef, pauli_string))
        The equivalent Hamiltonian for the generated graph
    weight_matrix : np.array
        Edge weights of the graph in matrix
    weights : np.array
        Edge weights of the graph in an array

    """

    coreset = [(w, v) for w, v in zip(coreset_weights, coreset_vectors)]

    if coreset is None:
        # Generate a graph instance with sample coreset data
        coreset = []
        # generate 3 points around x=-1, y=-1
        for _ in range(3):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                    np.random.normal(loc=-1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

        # generate 3 points around x=+1, y=1
        for _ in range(2):
            # use a uniformly random weight
            # weight = np.random.uniform(0.1,5.0,1)[0]
            weight = 1
            vector = np.array(
                [
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                    np.random.normal(loc=1, scale=0.5, size=1)[0],
                ]
            )
            new_point = (weight, vector)
            coreset.append(new_point)

    # Generate a networkx graph with correct edge weights
    n = len(coreset)
    G = nx.complete_graph(n)
    H = []
    weights = []
    weight_matrix = np.zeros(len(G.nodes) ** 2).reshape(len(G.nodes()), -1)
    for edge in G.edges():
        pauli_str = ["I"] * n
        # coreset points are labelled by their vertex index
        v_i = edge[0]
        v_j = edge[1]
        pauli_str[v_i] = "Z"
        pauli_str[v_j] = "Z"
        w_i = coreset[v_i][0]
        w_j = coreset[v_j][0]
        if metric == "dot":
            mval = np.dot(coreset[v_i][1], coreset[v_j][1])
        elif metric == "dist":
            mval = np.linalg.norm(coreset[v_i][1] - coreset[v_j][1])
        else:
            raise Exception("Unknown metric: {}".format(metric))

        weight_val = w_i * w_j
        weight_matrix[v_i, v_j] = weight_val
        weight_matrix[v_j, v_i] = weight_val
        G[v_i][v_j]["weight"] = w_i * w_j * mval
        weights.append(w_i * w_j * mval)
        H.append((w_i * w_j * mval, pauli_str))

    return coreset, G, H, weight_matrix, weights

In [5]:
G, weights, qubits = get_Hamil_variables(coreset_vectors, coreset_weights)
H = create_Hamiltonian_for_K2(G=G,qubits=qubits )

In [9]:
def kernel_two_local() -> cudaq.Kernel:
    """QAOA ansatz for maxcut"""
    kernel, thetas = cudaq.make_kernel(list)
    qreg = kernel.qalloc(number_of_qubits)

    # Create superposition
    kernel.h(qreg)

    # Loop over the layers
    theta_position = 0
    
    for i in range(layer_count):
        # Loop over the qubits
        # Problem unitary
        for j in range(1,number_of_qubits):
            kernel.rz(thetas[theta_position], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 1], qreg[j % number_of_qubits])
            kernel.cx(qreg[j], qreg[(j + 1) % number_of_qubits])
            kernel.rz(thetas[theta_position + 2], qreg[j % number_of_qubits])
            kernel.rx(thetas[theta_position + 3], qreg[j % number_of_qubits])
            theta_position += 4



    return kernel

In [15]:
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

Initial parameters =  [-0.11569440739371539, -0.383645704202297, -0.00830435745868574, 0.1236489312114839, 0.04072879367587334, 0.16545885016261164, -0.19465795167462596, -0.032768192483272285, -0.32568558523786784, 0.3300441503532814, -0.07228048728111086, 0.009162796908746551, 0.13057613921451494, 0.3358810785835822, 0.2411488995850365, -0.22220961558177665]


In [16]:
parameter_count

16

In [17]:
optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(),
    spin_operator=H,
    optimizer=optimizer,
    parameter_count=parameter_count,
    shots = 100)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_two_local(), optimal_parameters, shots_count = 100)


<H> = -0.001148
<H> = 0.013177
<H> = 0.005972
<H> = -0.008036
<H> = -0.049085
<H> = -0.025774
<H> = -0.042342
<H> = -0.056305
<H> = 0.139257
<H> = 0.118650
<H> = -0.137244
<H> = -0.133759
<H> = 0.003737
<H> = -0.123772
<H> = -0.127797
<H> = -0.147829
<H> = -0.057653
<H> = -0.150204
<H> = -0.083707
<H> = -0.121164
<H> = -0.144712
<H> = -0.280834
<H> = -0.074312
<H> = -0.216966
<H> = -0.243423
<H> = -0.286174
<H> = -0.159540
<H> = -0.283919
<H> = -0.164572
<H> = -0.248440
<H> = -0.419535
<H> = -0.104705
<H> = -0.415716
<H> = -0.190865
<H> = -0.478481
<H> = -0.500388
<H> = -0.477479
<H> = -0.066413
<H> = -0.543779
<H> = -0.438659
<H> = -0.595426
<H> = -0.294735
<H> = -0.576433
<H> = -0.133613
<H> = -0.563592
<H> = -0.314325
<H> = -0.588905
<H> = -0.513712
<H> = -0.556614
<H> = -0.447366
<H> = -0.566605
<H> = -0.620807
<H> = -0.589940
<H> = -0.402658
<H> = -0.645811
<H> = -0.230985
<H> = -0.648954
<H> = -0.226495
<H> = -0.646380
<H> = -0.584282
<H> = -0.622630
<H> = 0.048078
<H> = -0.58531

In [9]:
print(counts)

{ 11010:100 }

